In [1]:
import pandas as pd
import os
from shutil import copy
import geopy.distance
from math import log
from decimal import Decimal

In [2]:
data_root = "/home/gayan/Desktop/CMLARE/Data"
unprocessed_folder = "2018-05-08 to 2018-05-15"
processed_folder = data_root+"/Processed/selected/"+unprocessed_folder #Output folder for each file
links_file= data_root+"/Master Files"+"/mapped_links_precip.csv"  #List of the selected links

processed_file = data_root+"/Processed/"+unprocessed_folder + ".csv"
rootUnprocessed_folder = data_root+"/Un Processed/"
unprocessed_folder = rootUnprocessed_folder + unprocessed_folder

In [3]:
link_data = []
links = []
#test_file = processed_folder+"/PM_IG30028_15_201803180000_01.csv"
processed_data_all = []

In [4]:
directory = os.listdir(unprocessed_folder)
correct_files = []
for file in directory:
    splitted_name = file.split("_")[1]      
    if(splitted_name.endswith("28")):
        correct_files.append(file)
for file in correct_files:
    if not os.path.exists(processed_folder):
        os.makedirs(processed_folder)
    copy(src=unprocessed_folder+"/"+file,dst=processed_folder)

In [5]:
links = pd.read_csv(links_file)

In [6]:
def calcPathLength(processed_data):
#     print(processed_data)
    start_cord = (processed_data["XStart"],processed_data["YStart"])
    end_cord = (processed_data["XEnd"],processed_data["YEnd"])
    return geopy.distance.vincenty(start_cord, end_cord).km
# def formatDate(processed_data):
#     return processed_data["DateTime"].replace("-","").replace(" ","").replace(":","")[:-2]
def dBm_to_W(dBm):
    power = 10 ** ((float(dBm)-30)/10)
    return Decimal(power).to_eng_string()

def W_to_dBm(power):
    dBm = 30 + 10 * log(float(power), 10)
    return (dBm)
def calcAtt(row):
    return Decimal(row["PTAvg"]) - Decimal(row["PRAvg"])
def calcSLAttn(row):
    return Decimal(row["RSL_AVG"]) - Decimal(row["TSL_AVG"])

In [7]:
def process_file(data_file):
    global link_data
    link_data = pd.DataFrame(columns=["ID","DateTime","PrecipStation","PRmax","PRmin","PTmin","PTmax","PRAvg","PTAvg","PAttAvg","XStart","YStart","RSL_MIN","RSL_MAX","RSL_AVG","TSL_MIN","TSL_MAX","TSL_AVG","SLAttn","XEnd","YEnd","PathLength","distance","Frequency","FrequencyBand"])
#     data_file = processed_folder+"/PM_IG30028_15_201805050000_01.csv"
    link_data_df = pd.read_csv(data_file, skiprows=1)
    leftMerge = links.merge(link_data_df, left_on=links["source1_name"],right_on=link_data_df["ResourceName"],how="left")
    leftMerge = leftMerge.drop(columns = ["key_0"])
    rightMerge = leftMerge.merge(link_data_df, left_on=leftMerge["source2_name"],right_on=link_data_df["ResourceName"],how="left")
    rightMerge.dropna(subset=["ResourceName_x"])
    source1Data = rightMerge[["id_1","CollectionTime_x","nearest_precip_station","latitude1","longitude1","RSL_MIN_x","RSL_MAX_x","RSL_AVG_x","TSL_MIN_y","TSL_MAX_y","TSL_AVG_y","latitude2","longitude2","distance","frequency2","frequency_band"]]
    source2Data = rightMerge[["id_2","CollectionTime_y","nearest_precip_station","latitude2","longitude2","RSL_MIN_y","RSL_MAX_y","RSL_AVG_y","TSL_MIN_x","TSL_MAX_x","TSL_AVG_x","latitude1","longitude1","distance","frequency1","frequency_band"]]
    source1Data = source1Data.rename(columns={"id_1" : "ID","CollectionTime_x" : "DateTime","PrecipStation":"nearest_precip_station","latitude1" : "XStart","longitude1" : "YStart","RSL_MIN_x":"RSL_MIN","RSL_MAX_x":"RSL_MAX","RSL_AVG_x":"RSL_AVG","TSL_MIN_y":"TSL_MIN","TSL_MAX_y":"TSL_MAX","TSL_AVG_y":"TSL_AVG","latitude2":"XEnd","longitude2": "YEnd","distance":"distance","frequency2":"Frequency","frequency_band":"FrequencyBand"})
    source2Data = source2Data.rename(columns={"id_2" : "ID","CollectionTime_y" : "DateTime","PrecipStation":"nearest_precip_station","latitude2" : "XStart","longitude2" : "YStart","RSL_MIN_y":"RSL_MIN","RSL_MAX_y":"RSL_MAX","RSL_AVG_y":"RSL_AVG","TSL_MIN_x":"TSL_MIN","TSL_MAX_x":"TSL_MAX","TSL_AVG_x":"TSL_AVG","latitude1":"XEnd","longitude1": "YEnd","distance":"distance","frequency1":"Frequency","frequency_band":"FrequencyBand"})

    # source1Data = rightMerge[["frequency_band","CollectionTime_x","RSL_MIN_x","RSL_MAX_x","RSL_AVG_x","TSL_MIN_y","TSL_MAX_y","TSL_AVG_y","frequency2","latitude1","longitude1","latitude2","longitude2","Id_1"]]
    # source1Data = source1Data.rename(columns={"frequency_band":"FrequencyBand","frequency2":"Frequency","CollectionTime_x" :"DateTime","RSL_MIN_x":"Pmin","RSL_MAX_x":"Pmax","latitude1":"XStart","longitude1":"YStart","latitude2":"XEnd","longitude2":"YEnd","Id_1":"ID"})
    # source2Data = rightMerge[["frequency_band","CollectionTime_y","RSL_MIN_y","RSL_MAX_y","latitude2","longitude2","latitude1","longitude1","Id_2"]]
    # source2Data = source2Data.rename(columns={"frequency_band":"FrequencyBand","CollectionTime_y" :"DateTime","RSL_MIN_y":"Pmin","RSL_MAX_y":"Pmax","latitude2":"XStart","longitude2":"YStart","latitude1":"XEnd","longitude1":"YEnd","Id_2":"ID"})
    data = pd.concat([source1Data,source2Data])
    # data = data.dropna()
    data["PathLength"] = data.apply (lambda row: calcPathLength (row),axis=1)
    data["PRmax"] = data["RSL_MAX"].apply(lambda row: dBm_to_W(row))
    data["PRmin"] = data["RSL_MIN"].apply(lambda row: dBm_to_W(row))
    data["PRAvg"] = data["RSL_AVG"].apply(lambda row: dBm_to_W(row))
    data["PTmax"] = data["TSL_MAX"].apply(lambda row: dBm_to_W(row))
    data["PTmin"] = data["TSL_MIN"].apply(lambda row: dBm_to_W(row))
    data["PTAvg"] = data["TSL_AVG"].apply(lambda row: dBm_to_W(row))
    data["PAttAvg"] = data.apply(lambda row: calcAtt(row),axis=1)
    data["SLAttn"] = data.apply(lambda row: calcSLAttn(row),axis=1)

    link_data = pd.concat([link_data,data])

In [8]:
def process_all():
    global link_data
    link_data = pd.DataFrame(columns=["Frequency","DateTime","Pmin","Pmax","XStart","YStart","XEnd","YEnd","ID","PathLength"])
    directory = os.listdir(processed_folder)
    directory = list(directory)
    correct_file_paths = []
    for file in directory:
#         print(file)
        correct_file_paths.append(str(processed_folder)+"/"+file)
        process_file(str(processed_folder)+"/"+file)
        print("process Successfull")
#     print(directory)


In [ ]:
process_all()

In [ ]:
link_data.to_csv(processed_file)